<a href="https://colab.research.google.com/github/otitamario/sp-pa-gep/blob/main/experiments/Example5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the repository into Colab runtime
!git clone https://github.com/otitamario/sp-pa-gep.git

# Move into repo root
%cd sp-pa-gep

# Make sure Python sees the project root
import sys
sys.path.append(".")

Cloning into 'sp-pa-gep'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 173 (delta 72), reused 68 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (173/173), 2.88 MiB | 4.65 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/sp-pa-gep


In [2]:
import os
# =========================
# OUTPUT DIRECTORY (save plots here)
# =========================
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

In [3]:
import numpy as np
import src.benchkit as bk  # if you saved the library; otherwise paste needed functions
# if you pasted plot_metric_across_cases into notebook, import it directly

n = 100
x_star = np.zeros(n)
x0 = np.ones(n)
u = np.zeros(n)

N = 200
alpha_seq = np.array([1.0/(k+2) for k in range(N)], float)
r = 1.0
mu_values = [0.1, 1.0, 5.0]

def resolvent_strong_monotone(mu):
    def S(x, r):
        return x / (1.0 + r*mu)
    return S

def make_sppa_step(mu):
    S = resolvent_strong_monotone(mu)
    def step_fn(x, k):
        alpha = float(alpha_seq[k])
        u_k = S(x, r)
        x_new = alpha*u + (1.0-alpha)*u_k
        return x_new, {}
    return step_fn

def make_wppa_step(mu):
    S = resolvent_strong_monotone(mu)
    def step_fn(x, k):
        alpha = float(alpha_seq[k])
        u_k = S(x, r)
        x_new = alpha*x + (1.0-alpha)*u_k
        return x_new, {}
    return step_fn

def make_residual(mu):
    S = resolvent_strong_monotone(mu)
    def residual_fn(x):
        return float(np.linalg.norm(x - S(x, r)))
    return residual_fn

def error_fn(x):
    return float(np.linalg.norm(x - x_star))

# --- run all cases ---
logs_by_case = {}
summaries = []

for mu in mu_values:
    logs_sppa, sum_sppa = bk.run(
        method_name=f"SPPA μ={mu}",
        x0=x0,
        max_iter=N,
        stop_tol_step=1e-12,   # keep full length for fair plot
        step_fn=make_sppa_step(mu),
        residual_fn=make_residual(mu),
        error_fn=error_fn,
    )
    logs_wppa, sum_wppa = bk.run(
        method_name=f"WPPA μ={mu}",
        x0=x0,
        max_iter=N,
        stop_tol_step=1e-12,
        step_fn=make_wppa_step(mu),
        residual_fn=make_residual(mu),
        error_fn=error_fn,
    )

    logs_by_case[f"SPPA μ={mu}"] = logs_sppa
    logs_by_case[f"WPPA μ={mu}"] = logs_wppa
    summaries.extend([sum_sppa, sum_wppa])

# --- one figure per metric across all μ ---
bk.plot_metric_across_cases(
    logs_by_case=logs_by_case,
    metric="step",
    ylabel=r"$\|x_{n+1}-x_n\|$",
    title="Example 5.1: Step sizes (all μ)",
    outpath="figures/ex51_steps_all_mu.png",
)

bk.plot_metric_across_cases(
    logs_by_case=logs_by_case,
    metric="residual",
    ylabel=r"$R(x_n)$",
    title="Example 5.1: Residual (all μ)",
    outpath="figures/ex51_residual_all_mu.png",
)

bk.plot_metric_across_cases(
    logs_by_case=logs_by_case,
    metric="error",
    ylabel=r"$\|x_n-\bar{x}\|$",
    title="Example 5.1: Error (all μ)",
    outpath="figures/ex51_error_all_mu.png",
)

# --- one table for all μ (6 rows) ---
print(bk.latex_table(
    summaries,
    caption="CPU/accuracy summary for Example 5.1 (all μ).",
    label="tab:ex51_all_mu",
))

\begin{table}[!ht]
\centering
\begin{tabular}{lrrrrrr}
\hline
Method & Iters & Total (s) & Avg resolvent (s) & Avg residual (s) & Final $\|x_n-\bar x\|$ & Final $R(x_n)$\\
\hline
SPPA μ=0.1 & 200 & 0.0073 & 0.00001 & 0.00001 & 2.6198e-10 & 2.3816e-11 \\
WPPA μ=0.1 & 200 & 0.0076 & 0.00001 & 0.00001 & 8.5540e-08 & 7.7763e-09 \\
SPPA μ=1.0 & 38 & 0.0014 & 0.00001 & 0.00001 & 9.3282e-13 & 4.6641e-13 \\
WPPA μ=1.0 & 48 & 0.0018 & 0.00001 & 0.00001 & 8.8818e-13 & 4.4409e-13 \\
SPPA μ=5.0 & 17 & 0.0007 & 0.00001 & 0.00001 & 3.2821e-14 & 2.7351e-14 \\
WPPA μ=5.0 & 23 & 0.0008 & 0.00001 & 0.00001 & 2.5062e-13 & 2.0885e-13 \\
\hline
\end{tabular}
\caption{CPU/accuracy summary for Example 5.1 (all μ).}
\label{tab:ex51_all_mu}
\end{table}



In [4]:
# ─── 1. Configurar seu nome e email (só precisa rodar uma vez)
!git config --global user.name "otitamario"
!git config --global user.email "mario2013sj@gmail.com"

In [5]:
!git add .

In [6]:
# ─── 2. Método mais seguro e prático (usando cache de credencial) ────────────────

from google.colab import userdata

# Salve o token como segredo no Colab (muito recomendado!)
# Vá na barra lateral esquerda → chave de cadeado "Secrets"
# Adicione um segredo chamado: GITHUB_TOKEN
# Cole seu token lá e marque "Notebook access"

token = userdata.get('GITHUB_TOKEN')          # pega o segredo
username = "otitamario"               # seu username
repo = "sp-pa-gep"              # ex: meu-projeto

# Configura o remote com o token (https)
!git remote set-url origin https://{token}@github.com/{username}/{repo}.git

# Ou se ainda não tem remote configurado:
#!git remote add origin https://{token}@github.com/{username}/{repo}.git

In [7]:
!git commit -m "Example 5.1"

[main dc6eff0] Example 5.1
 3 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 figures/ex51_error_all_mu.png
 create mode 100644 figures/ex51_residual_all_mu.png
 create mode 100644 figures/ex51_steps_all_mu.png


In [8]:
!git pull origin main --rebase

From https://github.com/otitamario/sp-pa-gep
 * branch            main       -> FETCH_HEAD
Current branch main is up to date.


In [9]:
!git push origin main

Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 574.42 KiB | 13.36 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/otitamario/sp-pa-gep.git
   d07934a..dc6eff0  main -> main
